## 🔧 Setup & Imports

In [ ]:
# Standard library imports
import sys
from pathlib import Path
import time

# Add src to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

print(f"✅ Project root: {project_root}")
print(f"✅ Python version: {sys.version}")

In [ ]:
# Import our modules
from config.settings import Settings, SettingsError
from api.gemini_client import GeminiClient, GeminiError
from api.huggingface_client import HuggingFaceClient, HuggingFaceError
from api.materials_project_client import MaterialsProjectClient, MaterialsProjectError
from utils.logger import setup_logger
from utils.helpers import retry_with_backoff, save_json, load_json

print("✅ All imports successful!")

## ⚙️ Configuration Testing

In [ ]:
# Load settings
settings = Settings()
print("Configuration loaded:")
print(settings)

In [ ]:
# Validate all API keys
try:
    validation = settings.validate_all()
    print("✅ Validation successful!")
    for api, status in validation.items():
        print(f"  - {api}: {'✅' if status else '❌'}")
except SettingsError as e:
    print(f"❌ Validation failed:\n{e}")
    print("\n⚠️ Please add API keys to .env file (see API_SETUP_GUIDE.md)")

## 📝 Logging System Testing

In [ ]:
# Setup logger
logger = setup_logger(log_level=settings.log_level)

# Test different log levels
logger.info("ℹ️ This is an INFO message")
logger.warning("⚠️ This is a WARNING message")
logger.error("❌ This is an ERROR message (just testing!)")
logger.success("✅ This is a SUCCESS message")

print("\n✅ Logger configured! Check logs/app.log for file output.")

## 🤖 Google Gemini Testing

In [ ]:
# Initialize Gemini client
try:
    gemini = GeminiClient(settings.gemini_api_key)
    print("✅ Gemini client initialized")
except SettingsError as e:
    print(f"❌ Cannot initialize Gemini: {e}")
    gemini = None

In [ ]:
# Test text generation
if gemini:
    prompt = "Explain what a perovskite material is in 2-3 sentences."
    
    print(f"Prompt: {prompt}\n")
    print("Generating response...\n")
    
    start_time = time.time()
    response = gemini.generate_text(
        prompt=prompt,
        max_tokens=200,
        temperature=0.3
    )
    duration = time.time() - start_time
    
    print(f"Response ({duration:.2f}s):\n")
    print(response)
    print(f"\n✅ Gemini API working! ({len(response)} characters)")
else:
    print("⚠️ Gemini client not available - add API key to .env")

In [ ]:
# Test with system instruction
if gemini:
    response = gemini.generate_text(
        prompt="What are quantum dots?",
        max_tokens=150,
        temperature=0.5,
        system_instruction="You are a materials science expert. Explain concepts clearly and concisely."
    )
    
    print("Response with system instruction:\n")
    print(response)
else:
    print("⚠️ Skipping - Gemini not available")

## 🤗 Hugging Face Testing

In [ ]:
# Initialize Hugging Face client
try:
    hf = HuggingFaceClient(settings.hf_token)
    print("✅ Hugging Face client initialized")
except SettingsError as e:
    print(f"❌ Cannot initialize HuggingFace: {e}")
    hf = None

In [ ]:
# Test text generation (using fast model)
if hf:
    prompt = "Machine learning is"
    
    print(f"Prompt: {prompt}\n")
    print("Generating with GPT-2 (may take 20-30s if model is loading)...\n")
    
    start_time = time.time()
    response = hf.generate_text(
        prompt=prompt,
        model="gpt2",
        max_length=100,
        temperature=0.8
    )
    duration = time.time() - start_time
    
    print(f"Response ({duration:.2f}s):\n")
    print(response)
    print(f"\n✅ Hugging Face API working!")
else:
    print("⚠️ Hugging Face client not available - add token to .env")

In [ ]:
# Test embeddings generation
if hf:
    text = "Materials science is fascinating"
    
    print(f"Generating embeddings for: '{text}'\n")
    
    embeddings = hf.get_embeddings(
        text=text,
        model="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    print(f"Embeddings dimension: {len(embeddings)}")
    print(f"First 10 values: {embeddings[:10]}")
    print(f"\n✅ Embeddings generated successfully!")
else:
    print("⚠️ Skipping - Hugging Face not available")

## 🔬 Materials Project Testing

In [ ]:
# Initialize Materials Project client
try:
    mp = MaterialsProjectClient(
        settings.mp_api_key,
        enable_cache=settings.cache_enabled
    )
    print("✅ Materials Project client initialized")
except SettingsError as e:
    print(f"❌ Cannot initialize Materials Project: {e}")
    mp = None

In [ ]:
# Search by formula
if mp:
    formula = "Si"  # Silicon
    
    print(f"Searching for materials with formula: {formula}\n")
    
    start_time = time.time()
    results = mp.search_by_formula(formula)
    duration = time.time() - start_time
    
    print(f"Found {len(results)} materials ({duration:.2f}s)\n")
    
    # Display first 5 results
    print("Top 5 results:")
    for i, material in enumerate(results[:5], 1):
        mat_id = material.get('material_id', 'N/A')
        formula = material.get('formula_pretty', 'N/A')
        band_gap = material.get('band_gap', 'N/A')
        energy = material.get('energy_above_hull', 'N/A')
        
        print(f"{i}. {mat_id}: {formula}")
        print(f"   Band gap: {band_gap} eV")
        print(f"   Energy above hull: {energy} eV/atom\n")
    
    print("✅ Materials Project API working!")
else:
    print("⚠️ Materials Project client not available - add API key to .env")

In [ ]:
# Get properties for specific material
if mp and len(results) > 0:
    material_id = results[0]['material_id']
    
    print(f"Getting detailed properties for: {material_id}\n")
    
    properties = mp.get_material_properties(material_id)
    
    print("Properties:")
    for key, value in properties.items():
        print(f"  {key}: {value}")
    
    print("\n✅ Property retrieval working!")
else:
    print("⚠️ Skipping - no materials found or MP not available")

## 🔄 Combined Example: Multi-API Workflow

In [ ]:
# Example: Ask Gemini about a material, then look it up in Materials Project

if gemini and mp:
    print("🔬 Multi-API Research Workflow Example\n")
    print("=" * 60)
    
    # Step 1: Ask Gemini for a material suggestion
    print("\n1️⃣ Asking Gemini for material suggestion...\n")
    
    question = "What is a promising material for solar cell applications? Give me just the chemical formula."
    suggestion = gemini.generate_text(question, max_tokens=50, temperature=0.3)
    
    print(f"Gemini suggests: {suggestion}\n")
    
    # Step 2: Extract formula (simplified - just take first word)
    # In a real system, you'd use better parsing
    words = suggestion.split()
    potential_formulas = [w.strip('.,!?;:') for w in words if any(c.isupper() for c in w)]
    
    if potential_formulas:
        formula = potential_formulas[0]
        print(f"2️⃣ Extracted formula: {formula}\n")
        
        # Step 3: Look up in Materials Project
        print(f"3️⃣ Looking up '{formula}' in Materials Project...\n")
        
        try:
            materials = mp.search_by_formula(formula)
            
            if materials:
                print(f"✅ Found {len(materials)} materials!\n")
                
                # Show first result
                mat = materials[0]
                print("Top result:")
                print(f"  ID: {mat.get('material_id')}")
                print(f"  Formula: {mat.get('formula_pretty')}")
                print(f"  Band gap: {mat.get('band_gap')} eV")
                print(f"  Density: {mat.get('density')} g/cm³")
                
                # Step 4: Ask Gemini to analyze
                print("\n4️⃣ Asking Gemini to analyze properties...\n")
                
                analysis_prompt = f"""The material {mat.get('formula_pretty')} has:
                - Band gap: {mat.get('band_gap')} eV
                - Density: {mat.get('density')} g/cm³
                
                Is this suitable for solar cells? Explain briefly."""
                
                analysis = gemini.generate_text(analysis_prompt, max_tokens=200)
                
                print(f"Gemini's analysis:\n{analysis}")
                
                print("\n" + "=" * 60)
                print("✅ Multi-API workflow complete!")
                print("\nThis demonstrates how Phase 2+ will work:")
                print("  1. LLM generates ideas")
                print("  2. Database lookups validate them")
                print("  3. LLM analyzes results")
                print("  4. System iterates autonomously")
                
            else:
                print(f"No materials found for {formula}")
                
        except MaterialsProjectError as e:
            print(f"Error searching Materials Project: {e}")
    else:
        print("Couldn't extract formula from Gemini's response")
        
else:
    print("⚠️ This example requires both Gemini and Materials Project APIs")
    print("Please add API keys to .env and re-run")

## 💾 Helper Utilities Testing

In [ ]:
# Test JSON save/load
test_data = {
    "experiment": "Phase 1 Testing",
    "timestamp": time.time(),
    "results": [1, 2, 3, 4, 5]
}

# Save to data/results
output_path = project_root / "data" / "results" / "test_output.json"
save_json(test_data, output_path)
print(f"✅ Saved test data to: {output_path}")

# Load it back
loaded_data = load_json(output_path)
print(f"✅ Loaded data back: {loaded_data}")

print("\n✅ Helper utilities working!")

## 📊 Summary & Next Steps

In [ ]:
# Print summary
print("=" * 60)
print("🎉 PHASE 1 TESTING COMPLETE")
print("=" * 60)

print("\n✅ Successfully tested:")
print("  - Configuration system (.env loading)")
print("  - Logging system (console + file)")
print("  - Google Gemini API (text generation)")
print("  - Hugging Face API (inference + embeddings)")
print("  - Materials Project API (search + properties)")
print("  - Helper utilities (JSON, retry, etc.)")
print("  - Multi-API workflows")

print("\n📚 What you learned:")
print("  - How to use each API client")
print("  - How to chain multiple APIs together")
print("  - How the autonomous agent will work")

print("\n🚀 Ready for Phase 2!")
print("\nPhase 2 will add:")
print("  - Automated paper downloads from arXiv")
print("  - PDF parsing and knowledge extraction")
print("  - Paper relevance scoring")
print("  - Research database management")

print("\n💡 Next steps:")
print("  1. Review this notebook's output")
print("  2. Run: python scripts/test_all_apis.py")
print("  3. Check logs/app.log for detailed logs")
print("  4. Wait for Phase 2 prompt!")

print("\n" + "=" * 60)